In [ ]:
import json
import os
import sys
import time
import boto3
import botocore

import pandas as pd

bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
prompt_df = pd.read_csv("./Data/PromptQuestions_withGT.csv")
prompt_df.head()

In [ ]:
# Define the model
modelId = 'mistral.mistral-7b-instruct-v0:2'
accept = 'application/json'
contentType = 'application/json'

# Set the result dataframe
res_data = {
    'prompt': [],
    'response_1': [],
    'response_1_time': [],
    'response_2': [],
    'response_2_time': [],
    'response_3': [],
    'response_3_time': [],
    'response_4': [],
    'response_4_time': [],
    'response_5': [],
    'response_5_time': []
}
result_df = pd.DataFrame(res_data)

In [ ]:
for i in range(len(prompt_df)):
    prompt_message = """<s>[INST] {question} [/INST]""".format(question = prompt_df.iloc[i]['Prompt'])
    body = json.dumps({ 
        'prompt': prompt_message,
        'max_tokens': 500,
        'temperature': 0.5,
    })
    
    # Response_1
    start_time = time.time()
    response_1 = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                     modelId=modelId, 
                                     accept=accept, 
                                     contentType=contentType)
    response_body_1_pre = json.loads(response_1.get('body').read().decode('utf-8'))
    response_body_1 = response_body_1_pre.get('outputs')[0].get('text')
    end_time = time.time()  
    execution_time_1 = end_time - start_time  

    # Response_2
    start_time = time.time()
    response_2 = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                     modelId=modelId, 
                                     accept=accept, 
                                     contentType=contentType)
    response_body_2_pre = json.loads(response_2.get('body').read().decode('utf-8'))
    response_body_2 = response_body_2_pre.get('outputs')[0].get('text')
    end_time = time.time()  
    execution_time_2 = end_time - start_time  

    # Response_3
    start_time = time.time()
    response_3 = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                     modelId=modelId, 
                                     accept=accept, 
                                     contentType=contentType)
    response_body_3_pre = json.loads(response_3.get('body').read().decode('utf-8'))
    response_body_3 = response_body_3_pre.get('outputs')[0].get('text')
    end_time = time.time()  
    execution_time_3 = end_time - start_time 

    # Response_4
    start_time = time.time()
    response_4 = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                     modelId=modelId, 
                                     accept=accept, 
                                     contentType=contentType)
    response_body_4_pre = json.loads(response_4.get('body').read().decode('utf-8'))
    response_body_4 = response_body_4_pre.get('outputs')[0].get('text')
    end_time = time.time()  
    execution_time_4 = end_time - start_time 

    # Response_5
    start_time = time.time()
    response_5 = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                     modelId=modelId, 
                                     accept=accept, 
                                     contentType=contentType)
    response_body_5_pre = json.loads(response_5.get('body').read().decode('utf-8'))
    response_body_5 = response_body_5_pre.get('outputs')[0].get('text')
    end_time = time.time()  
    execution_time_5 = end_time - start_time 

    new_row = pd.DataFrame({
        'prompt': prompt_df.iloc[i]['Prompt'],
        'GroundTruth': prompt_df.iloc[i]['GroundTruth'],
        'response_1': response_body_1,
        'response_1_time': execution_time_1,
        'response_2': response_body_2,
        'response_2_time': execution_time_2,
        'response_3': response_body_3,
        'response_3_time': execution_time_3,
        'response_4': response_body_4,
        'response_4_time': execution_time_4,
        'response_5': response_body_5,
        'response_5_time': execution_time_5,
    },  index=[0])

    result_df =  pd.concat([result_df, new_row], ignore_index=True)
    
    time.sleep(15)

In [ ]:
result_df

In [ ]:
time_cols = ['response_1_time', 'response_2_time', 'response_3_time', 'response_4_time', 'response_5_time']

In [ ]:
result_df['avg_time'] = result_df[time_cols].apply(lambda x: x.mean(), axis=1)

result_df['median_time'] = result_df[time_cols].apply(lambda x: x.median(), axis=1)

In [ ]:
result_df.to_csv("./Results/Mistral-7b-with-gt-results-2.csv", index=False)